In [17]:
import fastf1
import os

# 1. SETUP PATHS
# We point specifically to 'data/raw' now to keep the project structured
cache_dir = os.path.join('..', 'data', 'raw')

# Safety check: ensure the folder exists
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

fastf1.Cache.enable_cache(cache_dir)
print(f"✅ FastF1 cache verified at: {os.path.abspath(cache_dir)}")

# 2. DATA EXTRACTION - The 'Light' Load
# Using 2024 Abu Dhabi (or 2025 if you have it) to verify the pipeline
year = 2024 
session_type = 'Q' # Qualifying

session_abd = fastf1.get_session(year, "Abu Dhabi", session_type)

# We load WITHOUT telemetry first to keep it fast
session_abd.load(telemetry=False, weather=False)

# 3. VERIFICATION
print(f"\n--- Connection Verified ---")
print(f"Event: {session_abd.event['EventName']} {year}")
print(f"Track: {session_abd.event['Location']}")
print(f"Drivers detected: {len(session_abd.results)} drivers on the grid.")

print("\n--- Step 1 Complete: Pipeline is connected and cached in /data/raw. ---")

✅ FastF1 cache verified at: c:\Users\trive\Desktop\vaanishka\Data Science\f1_pred\data\raw


core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INF


--- Connection Verified ---
Event: Abu Dhabi Grand Prix 2024
Track: Yas Island
Drivers detected: 20 drivers on the grid.

--- Step 1 Complete: Pipeline is connected and cached in /data/raw. ---


In [ ]:
import os
import json

# 1. FORCE the project root (Update this if your folder name is different)
PROJECT_NAME = "f1_pred"

# 2. Get the path to your current notebook
current_dir = os.getcwd()

# 3. Find where the project folder starts in your path
if PROJECT_NAME in current_dir:
    project_root = current_dir.split(PROJECT_NAME)[0] + PROJECT_NAME
else:
    # If we can't find it, we'll assume the current folder is the root for now
    project_root = current_dir

# 4. Construct the path to the rules file
# This goes: Root -> coefficients -> rules.json
rulebook_path = os.path.join(project_root, 'coefficient', 'rules.json')

print(f"--- PATH DEBUGGER ---")
print(f"Notebook is at: {current_dir}")
print(f"Project Root is: {project_root}")
print(f"Looking for rules at: {rulebook_path}")
print(f"---------------------")

# 5. The Moment of Truth
if os.path.exists(rulebook_path):
    with open(rulebook_path, 'r') as f:
        rules = json.load(f)
    print(f"✅ Success! Fuel penalty found: {rules['physics_constants']['fuel_weight_penalty_per_10kg']}")
else:
    print("❌ Still missing. Check your VS Code Sidebar:")
    print(f"1. Is there a folder named 'coefficient' inside '{PROJECT_NAME}'?")
    print(f"2. Is 'rules.json' inside that folder?")

--- PATH DEBUGGER ---
Notebook is at: c:\Users\trive\Desktop\vaanishka\Data Science\f1_pred\script
Project Root is: c:\Users\trive\Desktop\vaanishka\Data Science\f1_pred
Looking for rules at: c:\Users\trive\Desktop\vaanishka\Data Science\f1_pred\coefficient\rules.json
---------------------
✅ Success! Fuel penalty found: 0.3


In [16]:
import fastf1
import os

# Point to the new 'raw' folder
cache_dir = '../data/raw'
fastf1.Cache.enable_cache(cache_dir)

# This shows exactly where FastF1 is looking
print(f"FastF1 is currently caching to: {os.path.abspath(cache_dir)}")

FastF1 is currently caching to: c:\Users\trive\Desktop\vaanishka\Data Science\f1_pred\data\raw


In [ ]:
import fastf1
import os

# 1. Use the same path logic we just fixed for your rules.json
# This ensures we point to your existing 'data/raw' folder
current_dir = os.getcwd()
project_root = current_dir.split("f1_pred")[0] + "f1_pred"
cache_path = os.path.join(project_root, 'data', 'raw')

# 2. Enable the cache pointing to your EXISTING files
fastf1.Cache.enable_cache(cache_path)

# 3. Load the session
# Since the files are already there, this should take 1-2 SECONDS, not minutes.
session = fastf1.get_session(2024, 'Abu Dhabi', 'FP2')
session.load()

print("✅ Data linked successfully from the local cache!")

core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 2 [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INF

✅ Data linked successfully from the local cache!


In [25]:
import fastf1
import pandas as pd
import os

# 1. SETUP
os.makedirs('../data/cache', exist_ok=True)
fastf1.Cache.enable_cache('../data/cache') 

session = fastf1.get_session(2024, 'Abu Dhabi', 'FP2')
session.load(telemetry=True)

all_laps_list = []

# 2. THE SMART LOOP
# We use pick_drivers() to avoid the deprecation warning
for driver in session.drivers:
    driver_laps = session.laps.pick_drivers(driver)
    
    for _, lap in driver_laps.iterlaps():
        try:
            # get_telemetry() merges car_data and pos_data automatically
            # It includes the DRS column by default!
            tel = lap.get_telemetry()
            
            # Check if DRS was active at any point in this specific lap
            # DRS is usually 10, 12, or 14 for 'ON' and 8 for 'OFF' in raw data, 
            # but FastF1 simplifies this to 0/1 or even/odd.
            # A value >= 10 usually indicates DRS is active.
            has_drs = 1 if (tel['DRS'] >= 10).any() else 0
            
            # Create a copy of the lap series and add our new column
            lap_data = lap.copy()
            lap_data['DRS_Active'] = has_drs
            all_laps_list.append(lap_data)
            
        except Exception as e:
            # If telemetry fails, we still keep the lap but mark DRS as 0
            lap_data = lap.copy()
            lap_data['DRS_Active'] = 0
            all_laps_list.append(lap_data)

# 3. CONSOLIDATE
final_df = pd.DataFrame(all_laps_list)

# 4. SAVE
final_df.to_csv('../data/processed/abu_dhabi_clean.csv', index=False)
print("🚀 SUCCESS: 'abu_dhabi_clean.csv' created with synced DRS data!")

core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 2 [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '30', '43', '44', '55', '61', '63', '77', '81']


KeyboardInterrupt: 

In [7]:
session_abd.laps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 00:18:47.398000,VER,1,NaT,1.0,1.0,0 days 00:16:47.163000,NaT,NaT,0 days 00:00:47.515000,...,True,Red Bull Racing,0 days 00:16:47.163000,NaT,1,NaN,False,,False,False
1,0 days 00:20:10.723000,VER,1,0 days 00:01:23.325000,2.0,1.0,NaT,NaT,0 days 00:00:17.095000,0 days 00:00:36.151000,...,True,Red Bull Racing,0 days 00:18:47.398000,NaT,1,NaN,False,,False,True
2,0 days 00:22:08.762000,VER,1,0 days 00:01:58.039000,3.0,1.0,NaT,0 days 00:22:07.156000,0 days 00:00:25.374000,0 days 00:00:49.422000,...,True,Red Bull Racing,0 days 00:20:10.723000,NaT,1,NaN,False,,False,False
3,0 days 00:29:36.335000,VER,1,NaT,4.0,2.0,0 days 00:27:03.735000,NaT,NaT,0 days 00:00:47.150000,...,True,Red Bull Racing,0 days 00:22:08.762000,NaT,1,NaN,False,,False,False
4,0 days 00:30:59.212000,VER,1,0 days 00:01:22.877000,5.0,2.0,NaT,NaT,0 days 00:00:16.911000,0 days 00:00:35.892000,...,True,Red Bull Racing,0 days 00:29:36.335000,NaT,1,NaN,False,,False,True
